<a href="https://colab.research.google.com/github/kiyong21c/kaggle/blob/main/20220613_cat-in-dat_improve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 성능 개선
 - 피처 엔지니어링 : 피처 맞춤 인코딩, 피처 스케일링
 - 하이퍼 파라미터 최적화 : 그리드 서치

 - 1. 인코딩을 피처 특성에 맞게 적용해야 함
 
  > 이진 피처와 순서형 피처 ord_1, ord_2는 수작업 인코딩

  > 순서형 피처 ord_3, ord_4, ord_5는 ordinal 인코딩

  > 명목형 피처와 날짜 피처는 원-핫 인코딩

 - 2. 피처 스케일링 적용(피처 간 값의 범위를 일치)
 
  > 순서형 피처에만 피처 스케일링 적용

  > 이진 피처, 명목형 피처, 날짜 피처는 인코딩 후 이미 최솟값:0, 최댓값:1로 범위가 일치하기 때문에 스케일링 필요 없음

 - 3. 하이퍼파리미터 최적화

In [2]:
import pandas as pd

data_path = '/content/drive/MyDrive/Colab Notebooks/kaggle/input/cat-in-the-dat/'

train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

## 피처 엔지니어링 1 : 피처 맞춤 인코딩
 - 모든 피처를 일괄적으로 원-핫 인코딩 하는것 보다 피처 특성에 맞게 인코딩하면 성능 개선

#### 데이터 합치기

In [3]:
all_data = pd.concat([train, test])
all_data = all_data.drop('target', axis=1) # 타깃값 제거

#### 이진 피처 인코딩
 - bin_0, bin_1, bin_2 피처는 이미 0, 1로 구성
 - bin_3, bin_4 피처는 T, F, Y, N 으로 구성 : 각각 1, 0으로 변경

In [4]:
# Series 객체에 map() 함수를 호출하면, Series의 모든 원소에 적용해 결과 반환
all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

#### 순서형 피처 인코딩

In [5]:
ord_features = ['ord_0','ord_1','ord_2','ord_3','ord_4','ord_5']

for i, feature in enumerate(ord_features):
    print(f'{feature} 고윳값 : {all_data[feature].unique()}')

ord_0 고윳값 : [2 1 3]
ord_1 고윳값 : ['Grandmaster' 'Expert' 'Novice' 'Contributor' 'Master']
ord_2 고윳값 : ['Cold' 'Hot' 'Lava Hot' 'Boiling Hot' 'Freezing' 'Warm']
ord_3 고윳값 : ['h' 'a' 'i' 'j' 'g' 'e' 'd' 'b' 'k' 'f' 'l' 'n' 'o' 'c' 'm']
ord_4 고윳값 : ['D' 'A' 'R' 'E' 'P' 'K' 'V' 'Q' 'Z' 'L' 'F' 'T' 'U' 'S' 'Y' 'B' 'H' 'J'
 'N' 'G' 'W' 'I' 'O' 'C' 'X' 'M']
ord_5 고윳값 : ['kr' 'bF' 'Jc' 'kW' 'qP' 'PZ' 'wy' 'Ed' 'qo' 'CZ' 'qX' 'su' 'dP' 'aP'
 'MV' 'oC' 'RL' 'fh' 'gJ' 'Hj' 'TR' 'CL' 'Sc' 'eQ' 'kC' 'qK' 'dh' 'gM'
 'Jf' 'fO' 'Eg' 'KZ' 'Vx' 'Fo' 'sV' 'eb' 'YC' 'RG' 'Ye' 'qA' 'lL' 'Qh'
 'Bd' 'be' 'hT' 'lF' 'nX' 'kK' 'av' 'uS' 'Jt' 'PA' 'Er' 'Qb' 'od' 'ut'
 'Dx' 'Xi' 'on' 'Dc' 'sD' 'rZ' 'Uu' 'sn' 'yc' 'Gb' 'Kq' 'dQ' 'hp' 'kL'
 'je' 'CU' 'Fd' 'PQ' 'Bn' 'ex' 'hh' 'ac' 'rp' 'dE' 'oG' 'oK' 'cp' 'mm'
 'vK' 'ek' 'dO' 'XI' 'CM' 'Vf' 'aO' 'qv' 'jp' 'Zq' 'Qo' 'DN' 'TZ' 'ke'
 'cG' 'tP' 'ud' 'tv' 'aM' 'xy' 'lx' 'To' 'uy' 'ZS' 'vy' 'ZR' 'AP' 'GJ'
 'Wv' 'ri' 'qw' 'Xh' 'FI' 'nh' 'KR' 'dB' 'BE' 'Bb' 'mc' 'MC' 'tM' 'N

 - ord_0 피처는 이미 숫자 : 인코딩 필요X
 - ord_1, ord_2 : 숫자를 정해서 인코딩
 - ord_3 ~ ord_5 : 알파벳 순서로 인코딩

In [6]:
ord1dict = {'Novice':0, 'Contributor':1, 'Expert':2, 'Master':3, 'Grandmaster':4}
ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2, 'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}

all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

ord_features = ['ord_0','ord_1','ord_2','ord_3','ord_4','ord_5']

for i, feature in enumerate(ord_features):
    print(f'{feature} 고윳값 : {all_data[feature].unique()}')

ord_0 고윳값 : [2 1 3]
ord_1 고윳값 : [4 2 0 1 3]
ord_2 고윳값 : [1 3 5 4 0 2]
ord_3 고윳값 : ['h' 'a' 'i' 'j' 'g' 'e' 'd' 'b' 'k' 'f' 'l' 'n' 'o' 'c' 'm']
ord_4 고윳값 : ['D' 'A' 'R' 'E' 'P' 'K' 'V' 'Q' 'Z' 'L' 'F' 'T' 'U' 'S' 'Y' 'B' 'H' 'J'
 'N' 'G' 'W' 'I' 'O' 'C' 'X' 'M']
ord_5 고윳값 : ['kr' 'bF' 'Jc' 'kW' 'qP' 'PZ' 'wy' 'Ed' 'qo' 'CZ' 'qX' 'su' 'dP' 'aP'
 'MV' 'oC' 'RL' 'fh' 'gJ' 'Hj' 'TR' 'CL' 'Sc' 'eQ' 'kC' 'qK' 'dh' 'gM'
 'Jf' 'fO' 'Eg' 'KZ' 'Vx' 'Fo' 'sV' 'eb' 'YC' 'RG' 'Ye' 'qA' 'lL' 'Qh'
 'Bd' 'be' 'hT' 'lF' 'nX' 'kK' 'av' 'uS' 'Jt' 'PA' 'Er' 'Qb' 'od' 'ut'
 'Dx' 'Xi' 'on' 'Dc' 'sD' 'rZ' 'Uu' 'sn' 'yc' 'Gb' 'Kq' 'dQ' 'hp' 'kL'
 'je' 'CU' 'Fd' 'PQ' 'Bn' 'ex' 'hh' 'ac' 'rp' 'dE' 'oG' 'oK' 'cp' 'mm'
 'vK' 'ek' 'dO' 'XI' 'CM' 'Vf' 'aO' 'qv' 'jp' 'Zq' 'Qo' 'DN' 'TZ' 'ke'
 'cG' 'tP' 'ud' 'tv' 'aM' 'xy' 'lx' 'To' 'uy' 'ZS' 'vy' 'ZR' 'AP' 'GJ'
 'Wv' 'ri' 'qw' 'Xh' 'FI' 'nh' 'KR' 'dB' 'BE' 'Bb' 'mc' 'MC' 'tM' 'NV'
 'ih' 'IK' 'Ob' 'RP' 'dN' 'us' 'dZ' 'yN' 'Nf' 'QM' 'jV' 'sY' 'wu' 'SB'
 'UO' 'Mx' 'JX'

 - ord_3, ord_4, ord_5 : 알파펫 순서대로 인코딩
 - 사이킷런의 OrdinalEncoder 사용
 - 알파벳순서대로 map() 적용해도 되지만 고윳값 개수가 많아 번거롭다

In [7]:
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3', 'ord_4', 'ord_5']

ord_encoder = OrdinalEncoder() # 인코더 객체 생성

all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345]) # 인코딩 적용

for i, feature in enumerate(ord_features):
    print(f'{feature} 고윳값 : {all_data[feature].unique()}')

ord_0 고윳값 : [2 1 3]
ord_1 고윳값 : [4 2 0 1 3]
ord_2 고윳값 : [1 3 5 4 0 2]
ord_3 고윳값 : [ 7.  0.  8.  9.  6.  4.  3.  1. 10.  5. 11. 13. 14.  2. 12.]
ord_4 고윳값 : [ 3.  0. 17.  4. 15. 10. 21. 16. 25. 11.  5. 19. 20. 18. 24.  1.  7.  9.
 13.  6. 22.  8. 14.  2. 23. 12.]
ord_5 고윳값 : [136.  93.  31. 134. 158.  53. 185.  17. 160.  11. 159. 170. 105.  90.
  41. 147.  60. 116. 117.  27.  66.   8.  65. 110. 129. 157. 108. 118.
  32. 115.  18.  35.  73.  22. 167. 111.  80.  59.  82. 155. 139.  57.
   6.  95. 120. 138. 145. 131.  92. 175.  33.  51.  19.  56. 151. 178.
  16.  79. 152.  15. 166. 163.  71. 169. 190.  25.  37. 106. 122. 132.
 127.  10.  21.  52.   7. 114. 121.  91. 165. 102. 148. 150. 100. 144.
 181. 113. 104.  77.   9.  72.  89. 161. 128.  86.  58.  14.  67. 135.
  97. 172. 176. 173.  88. 187. 141.  68. 180.  84. 183.  83.   0.  24.
  76. 164. 162.  78.  20. 146.  34. 101.   4.   5. 143.  39. 171.  45.
 124.  28.  49.  61. 103. 177. 107. 188.  46.  55. 126. 168. 184.  64.
  69.  44.  30.

 - 인코딩이 완료됨
 - 어떤순서로 인코딩 했는지 알고 싶다면 : ord_encoder.categories_

In [8]:
ord_encoder.categories_

[array(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
        'n', 'o'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
        'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
       dtype=object),
 array(['AP', 'Ai', 'Aj', 'BA', 'BE', 'Bb', 'Bd', 'Bn', 'CL', 'CM', 'CU',
        'CZ', 'Cl', 'DH', 'DN', 'Dc', 'Dx', 'Ed', 'Eg', 'Er', 'FI', 'Fd',
        'Fo', 'GD', 'GJ', 'Gb', 'Gx', 'Hj', 'IK', 'Id', 'JX', 'Jc', 'Jf',
        'Jt', 'KR', 'KZ', 'Kf', 'Kq', 'LE', 'MC', 'MO', 'MV', 'Mf', 'Ml',
        'Mx', 'NV', 'Nf', 'Nk', 'OR', 'Ob', 'Os', 'PA', 'PQ', 'PZ', 'Ps',
        'QM', 'Qb', 'Qh', 'Qo', 'RG', 'RL', 'RP', 'Rm', 'Ry', 'SB', 'Sc',
        'TR', 'TZ', 'To', 'UO', 'Uk', 'Uu', 'Vf', 'Vx', 'WE', 'Wc', 'Wv',
        'XI', 'Xh', 'Xi', 'YC', 'Yb', 'Ye', 'ZR', 'ZS', 'Zc', 'Zq', 'aF',
        'aM', 'aO', 'aP', 'ac', 'av', 'bF', 'bJ', 'be', 'cA', 'cG', 'cW',
        'ck', 'cp', 'dB', 'dE', 'dN', 'dO', 'dP', 'dQ', 'd

#### 명목형 피처 인코딩
 - 순서 무시해도 되므로 원-핫 인코딩 적용

In [9]:
# List Comprehension : 지능형 리스트
nom_features = ['nom_' + str(i) for i in range(10)]
print(nom_features)

['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4', 'nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']


 - 원-핫 인코딩하면 열 개수가 늘어나서 all_data에 곧바로 인코딩할 수 없다
 - 희소 행렬을 CSR 형식으로 반환한다
  > 원-핫 인코딩을 적용하면, 대부분 값이 0으로 채워진 '희소 행렬'을 만든다

  > 메모리 낭비, 행렬의 크기가 늘어나 연산시간 증가

  > 개선을 위해 COO, CSR 형식을 자동으로 사용

In [10]:
all_data[nom_features].shape

(500000, 10)

In [11]:
all_data[nom_features].head()

,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9
id,,,,,,,,,,
0,Green,Triangle,Snake,Finland,Bassoon,50f116bcf,3ac1b8814,68f6ad3e9,c389000ab,2f4cb3d51
1,Green,Trapezoid,Hamster,Russia,Piano,b3b4d25d0,fbcb50fc1,3b6dd5612,4cd920251,f83c56c21
2,Blue,Trapezoid,Lion,Russia,Theremin,3263bdce5,0922e3cb8,a6a36f527,de9c9f684,ae6800dd0
3,Red,Trapezoid,Snake,Canada,Oboe,f12246592,50d7ad46a,ec69236eb,4ade6ab69,8270f0d71
4,Red,Trapezoid,Lion,Canada,Oboe,5b0f5acd5,1fe17a1fd,04ddac2be,cb43ab175,b164b72a7


In [12]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder() # 인코더 객체 생성

encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])

encoded_nom_matrix

<500000x16276 sparse matrix of type '<class 'numpy.float64'>'
	with 5000000 stored elements in Compressed Sparse Row format>

 - 행500000x열16276(CSR형식) 인 이유
  > 'nom_0'피처의 Green, Blue, Red 세가지를 원-핫 인코딩 하기위해 3개의 새로운 열이 필요

  > 나머지 명목형 피처들도 unique한 개수만큼 새로운 열이 필요

In [13]:
all_data = all_data.drop(nom_features, axis=1) # 기존 명목형 피처 삭제, 추후에 encoded_num_matrix를 all_data에 붙일예정

#### 날짜 피처 인코딩
 - 원-핫 인코딩 적용

In [14]:
date_features = ['day','month'] # 날짜 피처

encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features]) # 인코딩 적용

all_data = all_data.drop(date_features, axis=1) # 기존 날짜형 피처 삭제

encoded_date_matrix

<500000x19 sparse matrix of type '<class 'numpy.float64'>'
	with 1000000 stored elements in Compressed Sparse Row format>

 - 원-핫 인코딩된 행렬 크기는(500,000 x 19)
  > day 피처 고윳값은 7개, month 피처 고윳값은 12개

## 피처 엔지니어링 2 : 피처 스케일링
 - 서로 다른 피처들의 값의 범위가 일치하도록 조정
 - 수치형 피처들의 유효 값 범위가 서로 다르면 훈련이 제대로 안됨
 - 이진/명목형/날짜 피처를 모두 0, 1로 인코딩함
 - 순서형 피처는 여러개의 값을 가지므로 0~1사이가 되도록 스케일링

#### 순서형 피처 스케일링
 - 다른 피처들과 범위를 맞추기 위해 min-max 정규화 적용 → 피처값의 범위를 0~1로 조정함

In [15]:
from sklearn.preprocessing import MinMaxScaler

ord_features = ['ord_' + str(i) for i in range(6)] # 순서형 피처

# min-max 정규화
all_data[ord_features] = MinMaxScaler().fit_transform(all_data[ord_features])

In [16]:
for i, feature in enumerate(ord_features):
    print(f'{feature} 고윳값 : {all_data[feature].unique()}')

ord_0 고윳값 : [0.5 0.  1. ]
ord_1 고윳값 : [1.   0.5  0.   0.25 0.75]
ord_2 고윳값 : [0.2 0.6 1.  0.8 0.  0.4]
ord_3 고윳값 : [0.5        0.         0.57142857 0.64285714 0.42857143 0.28571429
 0.21428571 0.07142857 0.71428571 0.35714286 0.78571429 0.92857143
 1.         0.14285714 0.85714286]
ord_4 고윳값 : [0.12 0.   0.68 0.16 0.6  0.4  0.84 0.64 1.   0.44 0.2  0.76 0.8  0.72
 0.96 0.04 0.28 0.36 0.52 0.24 0.88 0.32 0.56 0.08 0.92 0.48]
ord_5 고윳값 : [0.71204188 0.48691099 0.16230366 0.70157068 0.82722513 0.27748691
 0.96858639 0.08900524 0.83769634 0.05759162 0.83246073 0.89005236
 0.54973822 0.47120419 0.21465969 0.76963351 0.31413613 0.60732984
 0.61256545 0.14136126 0.34554974 0.04188482 0.34031414 0.57591623
 0.67539267 0.82198953 0.56544503 0.61780105 0.16753927 0.60209424
 0.09424084 0.18324607 0.38219895 0.11518325 0.87434555 0.58115183
 0.41884817 0.30890052 0.42931937 0.81151832 0.72774869 0.29842932
 0.03141361 0.4973822  0.62827225 0.72251309 0.7591623  0.68586387
 0.48167539 0.91623037 

 - ord_0 고윳값 : [2 1 3] → [0.5 0. 1]

#### 인코딩 및 스케일링된 피처 합치기
 - 아래의 세가지 피처 합치기
  > all_data : 인코딩된 이진 피처와 순서형 피처(Dataframe 형식)

  > encoded_nom_matrix : 인코딩된 명목형 피처(CSR 행렬)

  > encoded_date_matrix : 인코딩된 날짜형 피처(CSR 행렬)

 - DataFrame 형식 → CSR 형식 변환
 - 사이파이 제공 csr_matrix()

In [17]:
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data),
                               encoded_nom_matrix,
                               encoded_date_matrix],
                              format='csr') # 합친 결과를 CSR형식으로 반환(기본값은 COO형식)

In [18]:
all_data_sprs

<500000x16306 sparse matrix of type '<class 'numpy.float64'>'
	with 9163718 stored elements in Compressed Sparse Row format>

 - 500,000행, 16,306열로 구성
 - 이정도 크기를 DataFrame으로 처리하면 메모리 낭비 및 훈련 속도 저하
 - 따라서 DataFrame으로 변환하지 않고, CSR 형식으로 그대로 사용

#### 데이터 나누기
 - 마지막으로 훈련데이터와 테스트데이터 나눔

In [19]:
num_train = len(train) # 훈련 데이터 개수(DataFrame의 row개수)

X_train = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target']

In [20]:
from sklearn.model_selection import train_test_split

# 훈련데이터에서 검증데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size=0.1,
                                                      stratify=y, # 타깃값이 같은비율로 들어가게 나눔
                                                      random_state=10)

In [21]:
X_train.shape, X_valid.shape

((270000, 16306), (30000, 16306))

### 하이퍼파라미터 최적화
 - 그리드 서치 활용 : C, max_iter
 - C : 규제 강도 조절 파라미터, 작을수록 규제 강도 세짐
 - 평가지표 : ROC AUC

In [33]:
# 셀 실행 후 소요 시간 출력
# %%time 

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 생성
logistic_model = LogisticRegression()

# 하이퍼 파라미터 값 목록
lr_params = {'C':[0.1,0.125], 'max_iter':[800,900],
             'solver':['liblinear'], 'random_state':[42]}

# 그리드 서치 객체 생성
gridsearch_logistic_model = GridSearchCV(estimator=logistic_model,
                                         param_grid=lr_params,
                                         scoring='roc_auc', # 평가지표
                                         cv=5) # 교차검증까지 수행함 (5-fold)

# 그리드서치 수행
gridsearch_logistic_model.fit(X_train, y_train)

# 최적의 하이퍼파라미터 확인
print(gridsearch_logistic_model.best_params_)

{'C': 0.125, 'max_iter': 800, 'random_state': 42, 'solver': 'liblinear'}


 - 1.로지스틱 회귀 모델 생성
 - 2.그리드서치 모델 생성(로지스틱 회귀 모델 주입)
 - 3.그리드서치 훈련 수행
 - 4.**최적 하이퍼파라미터로 훈련된 그리드서치 모델로 갱신**
  > 모델.best_estimator_ 로 최적 하이퍼파라미터로 훈련 가능
  
  > 모델.best_estimator_ 하지 않아도 자동으로 갱신됨


 - 코드는 병렬로 실행되며, CPU 시간은 개별 코어의 수행시간을 모두 합친값
 - 최적 하이퍼파라미터 → C:0.125, max_iter:800
 - 최적 하이퍼파라미터로 훈련된 모델로 갱신됨

### 모델 성능 검증
 - 검증데이터로 모델 성능 검증

In [31]:
# 먼저 검증데이터로 타깃 예측값 구하기
y_valid_preds = gridsearch_logistic_model.predict_proba(X_valid)[:, 1] # 검증데이터의 타깃값:1 일 확률

In [32]:
# 검증데이터 ROC AUC
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'{roc_auc:.4f}')

0.8045


 - 베이스라인 모델보다 0.008만큼 향상됨

### 예측 및 결과 제출

In [38]:
# 타깃값 1일 확률 예측
y_preds = gridsearch_logistic_model.best_estimator_.predict_proba(X_test)[:, 1]
# .best_estimator_ : 사용하지 않아도 훈련시에 판단된 최적의 하이퍼파라미터가 자동적용됨

# 제출 파일 생성
submission['target'] = y_preds
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/kaggle/input/cat-in-the-dat/submission.csv')

 - 앞서 모델 훈련 시 전체 훈련 데이터 100% 중 검증 데이터 10%를 떼어두고 90%만으로 훈련했다
 - 적은 데이터 이용시 10%도 상당히 아까운 데이터
 - 훈련데이터 100%로 다시 모델링 하면 약간 점수가 상승한다 

# 모델 성능개선의 일반적인 흐름
 - 위에서는 하나의 모델(로지스틱 회귀)을 사용 했지만,
 - 일반적으로는 여러 가지 방법으로 모델링
 - 그 중 가장 높은 성능의 모델 선정
 - 선정된 모델을 검증 데이터까지 포함한 전체 훈련데이터로 다시 훈련